# Trend based strategy

In [1]:
# Import basic libraries for manipulating data.

# Please refer to xarray.pydata.org for xarray documentation.

# xarray works optimally with N-dimensional datasets in Python
# and is well suited for financial datasets with labels "time",
# "field" and "asset". xarray data structures can also be easily
# converted to pandas dataframes.

import xarray as xr
import xarray.ufuncs as xruf

import numpy as np
import pandas as pd

# Import quantnet libraries.

import qnt.data as qndata          # data loading and manipulation
import qnt.stats as qnstats        # key statistics
import qnt.graph as qngraph        # graphical tools
import qnt.forward_looking as qnfl # forward looking checking
import qnt.xr_talib as xrtl        # technical analysis indicators (talib)
import qnt.output as qnout

# display function for fancy displaying:
from IPython.display import display
# lib for charts
import plotly.graph_objs as go

In [2]:
# Load all available data since given date.

# It is possible to set a max_date in the call in order to
# develop the system on a limited in-sample period and later
# test the system on unseen data after max_date.

# A submission will be accepted only if no max_date is set,
# as submissions will be evaluated on live data on a daily basis.

data = qndata.futures.load_data(tail=10*365, dims=("time", "field", "asset"))

We will use WMA and ROCP from qnt.xr_talib to measure trend.

In [3]:
help(xrtl.WMA)

Help on function WMA in module qnt.xr_talib:

WMA(data: xarray.core.dataarray.DataArray, timeperiod: int = 30) -> xarray.core.dataarray.DataArray
    Weighted Moving Average (Overlap Studies)
    Parameters:
        timeperiod: 30
    Input:
        data: time series
    Output:
        double series



In [4]:
help(xrtl.ROCP)

Help on function ROCP in module qnt.xr_talib:

ROCP(data: xarray.core.dataarray.DataArray, timeperiod: int = 14) -> xarray.core.dataarray.DataArray
     Rate of change Percentage: (real-prevPrice)/prevPrice (Momentum Indicators)
    Input:
        data: time series
    Parameters:
        timeperiod: 14
    Output:
        double series



Let's implement strategy based on WMA using one asset:

In [5]:
stock_name = 'GC'

# select only 1 stock
stock = data.sel(asset=stock_name).dropna('time', 'all')

pd_time = stock.time.to_pandas()
close = stock.sel(field='close')

# chart with prices
price_fig = [
   go.Candlestick(
       x=stock.time.to_pandas(),
       open=stock.sel(field='open').values,
       high=stock.sel(field='high').values,
       low=stock.sel(field='low').values,
       close=stock.sel(field='close').values,
       name=stock_name
   )
]

# calculate MA 
ma = xrtl.WMA(close, timeperiod=16) # you can use also SMA, EMA, etc.
# calcuate ROC
roc = xrtl.ROCP(ma, timeperiod=2)

# We suppose, when abs(roc) < sideways_threshold, the trend is sideways. 
sideways_threshold = 0.01

# positive trend direction
positive_trend = roc > sideways_threshold 
# negtive trend direction
negative_trend = roc < -sideways_threshold 
# sideways
sideways_trend = abs(roc) <= sideways_threshold

# This is a street magic. We will elliminate sideway
# We suppose that a sideways trend after a positive trend is also positive
side_positive_trend = positive_trend.where(sideways_trend == False).ffill('time').fillna(False)
# and a sideways trend after a negative trend is also negative
side_negative_trend = negative_trend.where(sideways_trend == False).ffill('time').fillna(False)

# charts with trend indicator

trend_fig = [
    go.Scatter(
        x = pd_time,
        y = ma,
        name='ma',
        line = dict(width=1,color='orange')
    ),
    go.Scatter(
        x = pd_time,
        y = ma.where(side_positive_trend),
        name='side-positive-trend',
        line = dict(width=1,color='green')
    ),
    go.Scatter(
        x = pd_time,
        y = ma.where(side_negative_trend),
        name='side-negative-trend',
        line = dict(width=1,color='red')
    ),
    go.Scatter(
        x = pd_time,
        y = ma.where(positive_trend),
        name='positive-trend',
        line = dict(width=3,color='green')
    ),
    go.Scatter(
        x = pd_time,
        y = ma.where(negative_trend),
        name='negative-trend',
        line = dict(width=3,color='red')
    ) 
]


# define signals
buy_signal = positive_trend
buy_stop_signal = side_negative_trend

sell_signal = negative_trend
sell_stop_signal = side_positive_trend

# calc positions 
position = close.copy(True)
position[:] = np.nan
position = xr.where(buy_signal, 1, position)
position = xr.where(sell_signal, -1, position)
position = xr.where(xruf.logical_and(buy_stop_signal, position.ffill('time') > 0), 0, position)
position = xr.where(xruf.logical_and(sell_stop_signal, position.ffill('time') < 0), 0, position)

position = position.ffill('time').fillna(0)

# calc real orders
real_buy = xruf.logical_and(position > 0, position.shift(time=1) <= 0)
real_sell = xruf.logical_and(position < 0, position.shift(time=1) >= 0)
real_stop = xruf.logical_and(position == 0, position.shift(time=1) != 0)

# plot orders chart
signals_fig=[
    go.Scatter(
        x=close.loc[real_buy].time.to_pandas(),
        y=close.loc[real_buy],
        mode="markers",
        hovertext='buy',
        name="buy",
        marker_size=9,
        marker_color='green'
    ),
    go.Scatter(
        x=close.loc[real_sell].time.to_pandas(),
        y=close.loc[real_sell],
        mode="markers",
        hovertext='sell',
        name="sell",
        marker_size=9,
        marker_color='red'
    ),
    go.Scatter(
        x=close.loc[real_stop].time.to_pandas(),
        y=close.loc[real_stop],
        mode="markers",
        hovertext='stop',
        name="stop",
        marker_size=9,
        marker_color='gray'
    ),
]

# draw chart
fig = go.Figure(data = price_fig + trend_fig + signals_fig)
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

# calc stats
position_with_asset = xr.concat([position], pd.Index([stock_name], name='asset'))
stats = qnstats.calc_stat(data, position_with_asset)
display(stats.to_pandas().tail())

performance = stats.loc[:,"equity"]

# draw performance chart
fig = go.Figure(data = [
    go.Scatter(
        x=performance.time.to_pandas(),
        y=performance,
        hovertext='performance',
    )
])
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-12-10,0.400206,0.000546,0.161122,-0.672102,-0.678497,-0.529822,-0.085366,-1.0,1.0,0.047536,29.802469
2020-12-11,0.398859,-0.003365,0.161094,-0.673205,-0.678497,-0.531582,-0.085635,-1.0,1.0,0.047519,29.802469
2020-12-14,0.401376,0.006311,0.161076,-0.671143,-0.678497,-0.527967,-0.085043,-1.0,1.0,0.047502,29.802469
2020-12-15,0.396230,-0.012821,0.161092,-0.675359,-0.678497,-0.534847,-0.086159,-1.0,1.0,0.047487,29.802469
2020-12-16,0.395410,-0.002069,0.161061,-0.676031,-0.678497,-0.535892,-0.086312,-1.0,1.0,0.047475,29.585366


Now, implement the strategy on multiple assets.

In [6]:
close = data.sel(field='close')

# trend
ma = xrtl.WMA(close, timeperiod=8)
roc = xrtl.ROCP(ma, timeperiod=1)

sideways_threshold = 0.01

positive_trend = roc > sideways_threshold 
negative_trend = roc < -sideways_threshold 
sideways_trend = abs(roc) <= sideways_threshold 

side_positive_trend = positive_trend.where(sideways_trend == False).ffill('time').fillna(False)
side_negative_trend = negative_trend.where(sideways_trend == False).ffill('time').fillna(False)

# signals
buy_signal = positive_trend
buy_stop_signal = side_negative_trend

sell_signal = negative_trend
sell_stop_signal = side_positive_trend

# calc positions 
position = close.copy(True)
position[:] = np.nan

# align signals
buy_signal = xr.align(buy_signal, position, join='right')[0] 
buy_stop_signal = xr.align(buy_stop_signal, position, join='right')[0]
sell_signal = xr.align(sell_signal, position, join='right')[0]
sell_stop_signal = xr.align(sell_stop_signal, position, join='right')[0]

# apply signals to position
position = xr.where(buy_signal, 1, position)
position = xr.where(sell_signal, -1, position)

fp = position.ffill('time')
position = xr.where(xruf.logical_and(buy_stop_signal, fp > 0), 0, position)
position = xr.where(xruf.logical_and(sell_stop_signal, fp < 0), 0, position)

position = position.ffill('time').fillna(0)


# position normalization
output = position/abs(position).sum('asset')


#calc and print stats
stats = qnstats.calc_stat(data, output)
display(output.to_pandas().tail())
display(stats.to_pandas().tail())

asset,A6,AE,AH,B6,CB,CC,CF,CL,CT,D6,...,ZF,ZL,ZM,ZN,ZO,ZQ,ZR,ZS,ZT,ZW
time,,,,,,,,,,,,,,,,,,,,,
2020-12-10,0.015873,0.015873,-0.015873,0.015873,0.015873,-0.015873,0.015873,0.015873,0.015873,-0.015873,...,0.0,0.015873,0.015873,0.0,0.015873,0.0,-0.015873,0.015873,0.0,0.015873
2020-12-11,0.015873,0.015873,-0.015873,0.015873,0.015873,-0.015873,0.015873,0.015873,0.015873,-0.015873,...,0.0,0.015873,0.015873,0.0,0.015873,0.0,-0.015873,0.015873,0.0,0.015873
2020-12-14,0.015873,0.015873,-0.015873,0.015873,0.015873,-0.015873,0.015873,0.015873,0.015873,-0.015873,...,0.0,0.015873,0.015873,0.0,0.015873,0.0,-0.015873,0.015873,0.0,0.015873
2020-12-15,0.015873,0.015873,-0.015873,0.015873,0.015873,-0.015873,0.015873,0.015873,0.015873,-0.015873,...,0.0,0.015873,0.015873,0.0,0.015873,0.0,-0.015873,0.015873,0.0,0.015873
2020-12-16,0.015873,0.015873,-0.015873,0.015873,0.015873,-0.015873,0.015873,0.015873,0.015873,-0.015873,...,0.0,0.015873,0.015873,0.0,0.015873,0.0,-0.015873,0.015873,0.0,0.015873


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-12-10,0.887727,0.003857,0.064569,-0.139932,-0.277808,-0.178675,-0.011537,0.555556,63.0,0.039457,60.163090
2020-12-11,0.886672,-0.001188,0.064558,-0.140953,-0.277808,-0.180410,-0.011647,0.523810,63.0,0.039446,60.163090
2020-12-14,0.887460,0.000889,0.064546,-0.140190,-0.277808,-0.179050,-0.011557,0.555556,63.0,0.039446,60.146605
2020-12-15,0.891389,0.004427,0.064548,-0.136384,-0.277808,-0.172389,-0.011127,0.555556,63.0,0.039446,60.133826
2020-12-16,0.893461,0.002325,0.064540,-0.134376,-0.277808,-0.168883,-0.010900,0.555556,63.0,0.039434,64.729385


## Improvement #1

As you can see, the result is not good. This strategy does not work for all assets all time. 
Well, let's try to find the top of assets with good "sharpe_ratio".

In [7]:
top_period = 30
top_size = 5

# normalize output per asset
output_per_asset = output/abs(output)

# calculate stats per asset
stats_per_asset = qnstats.calc_stat(data, output_per_asset, per_asset=True, max_periods=top_period)

# calculate ranks of assets by "sharpe_ratio"
ranks = (-stats_per_asset.sel(field='sharpe_ratio')).rank('asset')
# Select top assets by rank which assets have 'top_period' days ago.
# We assume, that the "sharpe_ratio" of these assets will be good in the next 'top_period' days
rank = ranks.isel(time = -top_period)
top = rank.where(rank <= top_size).dropna('asset').asset

# select top stats
top_stats = stats_per_asset.sel(asset = top.values)

# print results
print("SR tail of the top assets:")
display(top_stats.sel(field='sharpe_ratio').to_pandas().tail())

print("avg SR = ", top_stats[-top_period:].sel(field = 'sharpe_ratio').mean('asset')[-1].item())

SR tail of the top assets:


asset,T6,ZC,ZM,ZS,ZW
time,,,,,
2020-12-10,9.257418,2.285306,0.236458,7.297212,-0.800841
2020-12-11,6.813549,2.634062,0.430754,7.429704,0.456287
2020-12-14,7.470783,2.893866,1.275822,8.854943,-0.877581
2020-12-15,7.727406,2.436268,1.993966,8.977001,-0.761031
2020-12-16,6.638871,2.151488,1.462952,6.785819,-0.730490


avg SR =  3.261727937877241


The results is good. Now, let's optimize entire output, using this idea:

In [8]:
top_period = 60
top_size = 10
top_step = 60

output_per_asset = output/abs(output)
stats_per_asset = qnstats.calc_stat(data, output_per_asset, per_asset=True, max_periods=top_period)
ranks = (-stats_per_asset.sel(field='sharpe_ratio')).rank('asset')

top_output = output.copy(True)
top_output[:] = 0

for offset in range(top_period - 1, len(ranks), top_step):
    start_date = ranks.time[offset].values
    end_date = ranks.time[min(offset + top_step - 1, len(ranks.time) - 1)].values
    rank = ranks.loc[start_date]
    top = rank.where(rank <= top_size).dropna('asset').asset
    top_output.loc[start_date:end_date, top] = output.loc[start_date:end_date, top]
    
# normalization
top_output = top_output / abs(top_output).sum('asset')
    
#calc stat
top_stats = qnstats.calc_stat(data, top_output)

# display stat
display(top_stats.to_pandas().tail())


# draw performance chart
performance = top_stats.loc[:,"equity"]
fig = go.Figure(data = [
    go.Scatter(
        x=performance.time.to_pandas(),
        y=performance,
        hovertext='performance',
    )
])
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-12-10,0.710105,0.010760,0.102389,-0.308028,-0.419816,-0.320416,-0.032807,0.4,61.0,0.065750,34.391967
2020-12-11,0.714489,0.006175,0.102388,-0.303755,-0.419816,-0.314631,-0.032214,0.4,61.0,0.065732,34.391967
2020-12-14,0.717760,0.004578,0.102378,-0.300568,-0.419816,-0.310335,-0.031772,0.4,61.0,0.065710,34.391967
2020-12-15,0.719870,0.002940,0.102363,-0.298512,-0.419816,-0.307561,-0.031483,0.4,61.0,0.065690,34.391967
2020-12-16,0.723906,0.005606,0.102359,-0.294579,-0.419816,-0.302309,-0.030944,1.0,61.0,0.065668,34.412568


### Statistics

In [9]:
# Calculate statistics on a rolling basis.

# Transactions are punished with slippage equal to a given
# fraction of ATR14 (read more about slippage in our full
# Strategy Buy and Hold template). We evaluate submissions
# using 5% of ATR14 for slippage.

# Mean return, volatility and Sharpe ratio are computed on a
# rolling basis using a lookback period of 7 years.

stat = qnstats.calc_stat(data, top_output)

display(stat.to_pandas().tail())

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-12-10,0.710105,0.010760,0.102389,-0.308028,-0.419816,-0.320416,-0.032807,0.4,61.0,0.065750,34.391967
2020-12-11,0.714489,0.006175,0.102388,-0.303755,-0.419816,-0.314631,-0.032214,0.4,61.0,0.065732,34.391967
2020-12-14,0.717760,0.004578,0.102378,-0.300568,-0.419816,-0.310335,-0.031772,0.4,61.0,0.065710,34.391967
2020-12-15,0.719870,0.002940,0.102363,-0.298512,-0.419816,-0.307561,-0.031483,0.4,61.0,0.065690,34.391967
2020-12-16,0.723906,0.005606,0.102359,-0.294579,-0.419816,-0.302309,-0.030944,1.0,61.0,0.065668,34.412568


In [10]:
def print_stat(stat):
    """Prints selected statistical key indicators:
       - the global Sharpe ratio of the strategy;
       - the global mean profit;
       - the global volatility;
       - the maximum drawdown.

       Note that Sharpe ratio, mean profit and volatility
       apply to  max simulation period, and not to the
       rolling basis of 3 years.
    """

    days = len(stat.coords["time"])
    
    returns = stat.loc[:, "relative_return"]

    equity = stat.loc[:, "equity"]
    
    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]

    profit = (qnstats.calc_mean_return_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    volatility = (qnstats.calc_volatility_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    max_ddown = (qnstats.calc_max_drawdown(
        qnstats.calc_underwater(equity)).to_pandas().values[-1])*100.0

    print("Sharpe Ratio         : ", "{0:.3f}".format(sharpe_ratio))
    print("Mean Return [%]      : ", "{0:.3f}".format(profit))
    print("Volatility [%]       : ", "{0:.3f}".format(volatility))
    print("Maximum Drawdown [%] : ", "{0:.3f}".format(-max_ddown))

print_stat(stat)

Sharpe Ratio         :  -0.306
Mean Return [%]      :  -3.179
Volatility [%]       :  10.378
Maximum Drawdown [%] :  41.982


In [11]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

In [12]:
# show underwater chart:
UWchart = stat.to_pandas()["underwater"].iloc[(252*3):]
qngraph.make_plot_filled(UWchart.index, UWchart, color="darkred", name="Underwater Chart", range_max=0)

In [13]:
# show bias chart:
biaschart = stat.to_pandas()["bias"]
qngraph.make_plot_filled(biaschart.index, biaschart, color="#5A6351", name="Bias Chart")

In [14]:
# show rolling Sharpe ratio on a 7-year basis:
SRchart = stat.to_pandas()["sharpe_ratio"]
qngraph.make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="Rolling SR")

### Checks

In [15]:
# check your output
qnout.check(output, data)

Check missed dates...
Ok.
Check sharpe ratio.


ERROR! The sharpe ratio is too low. -0.17119561595202581 < 1


Check correlation.

Ok. This strategy does not correlate with other strategies.


## Write output

In [16]:
# Finally, we write the last mandatory step for submission,
# namely writing output to file:

qnout.write(output)

Write output: /root/fractions.nc.gz


At this stage code is ready for submission. Just click on the submission button in your account page and we will evaluate your strategy live on our servers!